In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential


import pathlib


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)



Mounted at /content/gdrive


In [3]:
dataset_url = "/content/gdrive/MyDrive/test_set"
data_dir = pathlib.Path(dataset_url)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

4684


In [26]:
batch_size = 16
img_height = 400
img_width = 300

train_X = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_X = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

#val_X, test_X = tf.keras.utils.split_dataset(val_X, left_size=0.5)

class_names = train_X.class_names
print(class_names)

Found 4684 files belonging to 8 classes.
Using 3748 files for training.
Found 4684 files belonging to 8 classes.
Using 936 files for validation.
['birch', 'hornbeam', 'liriodendron', 'maple', 'red_hornbeam', 'royal_red_maple', 'silver_linden', 'small_leaved_linden']


In [31]:
data_augmentation = keras.Sequential(
  [  
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomBrightness([-0.2,0.2]),
    layers.RandomContrast(0.2),
  ]
)


normalization_layer = layers.Rescaling(1./255)

num_classes = len(class_names)
rescaler = layers.Rescaling(1./255)

In [27]:
model1 = Sequential([ # Accuracy ~0.9
  data_augmentation,
  rescaler,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])



In [36]:
model3 = Sequential([ 
  data_augmentation,
  rescaler,
  layers.Conv2D(16, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs", activation = 'softmax')


])

In [37]:
model3.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#model.summary()

epochs=100
history = model3.fit(
  train_X,
  validation_data=val_X,
  epochs=epochs,
  verbose='auto',
  validation_freq=[5,10,15,20,25,30,35,40,45,50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
)

Epoch 1/100
235/235 [==============================] - 24s 89ms/step - loss: 1.6735 - accuracy: 0.3559
Epoch 2/100
235/235 [==============================] - 21s 87ms/step - loss: 1.1221 - accuracy: 0.5672
Epoch 3/100
235/235 [==============================] - 21s 86ms/step - loss: 0.8539 - accuracy: 0.6980
Epoch 4/100
235/235 [==============================] - 21s 87ms/step - loss: 0.7467 - accuracy: 0.7292
Epoch 5/100
235/235 [==============================] - 27s 114ms/step - loss: 0.6499 - accuracy: 0.7721 - val_loss: 0.5539 - val_accuracy: 0.8152
Epoch 6/100
235/235 [==============================] - 21s 86ms/step - loss: 0.6652 - accuracy: 0.7511
Epoch 7/100
235/235 [==============================] - 21s 88ms/step - loss: 0.5911 - accuracy: 0.7908
Epoch 8/100
235/235 [==============================] - 22s 92ms/step - loss: 0.5576 - accuracy: 0.8026
Epoch 9/100
235/235 [==============================] - 22s 88ms/step - loss: 0.4877 - accuracy: 0.8279
Epoch 10/100
235/235 [========